<a href="https://colab.research.google.com/github/ElUnrast/GermanWave2Vec/blob/main/Wave2Vec_ah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Configuartion

In [1]:
import os
import sys

device = 'cuda'
wait_for_v100 = False
ds_to_us = ['HP1-FvM']
home = os.getenv("HOME")
use_gdrive = False
trained_model_directory = 'trained_model_ah'
WANDB_API_KEY='6cec6a71175dec59b32a9067a0664dec76208f2a'
WANDB_NAME='GermanWaveToVec Training'
WANDB_ENTITY='wandb.ai/elunrast'
WANDB_PROJECT='GermanWaveToVec'
# Use Model from Checkpoint. 
# The Value must be a Sub-Directory of {trained_model_path} (Example: 'checkpoint-3600')
checkpoint = None

## ----- Training arguments  ----- ##
per_device_train_batch_size = 5      # TODO: aus freiem Grafikkarten Speicher ableiten (Graphikkarte mit 11GB)
per_device_eval_batch_size  = per_device_train_batch_size // 2
gradient_accumulation_steps = 2
logging_steps               = 100
max_steps                   = per_device_train_batch_size * logging_steps
max_trainingset_size        = max_steps * per_device_train_batch_size # TODO freiem Grafikkarten Speicher ableiten
max_steps                   = max_trainingset_size // per_device_train_batch_size # Schritte pro Epoche
warmup_steps                = max_steps // 4
save_steps                  = max_steps // 2
eval_steps                  = save_steps
max_sample_size             = 1200   # None -> no restriction
max_trainingset_size        = per_device_train_batch_size * max_steps   # None -> no restriction
fixed_training_set_size     = 100 * per_device_eval_batch_size          # None -> 20% der Originaldatensatz Größe 
num_train_epochs            = 1  # Anzahl Durchläufe für ein Dataset
max_rounds                  = 1  # Anzahl Trainingsrunden über alle used_datasets

learning_rate = per_device_train_batch_size * ((3e-4)/16)  # 3e-4/4 

## 1. Setup Directories

In [2]:
local_nlp_base_dir = f'{home}/shared'
local_data_base_dir = f'{local_nlp_base_dir}/NLP-Data'
local_data_dir = f'{local_data_base_dir}/audio'
extern_nlp_base_dir = None
extern_data_base_dir = None

runs_on_colab = (home == '/root')
print( f'runs on colab: {runs_on_colab}')

if not runs_on_colab:
    os.environ['http_proxy'] = 'http://192.168.8.50:3128'
    os.environ['https_proxy'] = 'http://192.168.8.50:3128'
else:
    # to get access to the datasets we use gdrive
    use_gdrive = True
    # install packages
    !pip install datasets==1.4.1
    !pip install transformers==4.4.0
    !pip install jiwer
    !pip install torchaudio
    !pip install librosa
    # create local directories
    !mkdir $local_nlp_base_dir
    !mkdir $local_data_base_dir

if use_gdrive:
    gdrive_base = '/content/gdrive'    
    extern_nlp_base_dir = f'{gdrive_base}/MyDrive'
    extern_data_base_dir = f'{extern_nlp_base_dir}/NLP-Data'
    extern_data_dir = f'{extern_data_base_dir}/audio'

    if not os.path.isdir(gdrive_base):
        from google.colab import drive
        drive.mount(gdrive_base)

if not os.path.isdir(local_data_dir):
    !mkdir $local_data_dir

if extern_nlp_base_dir:
    model_dir = f'{extern_nlp_base_dir}/NLP-Models/GermanWave2Vec'
else:
    model_dir = f'{local_nlp_base_dir}/NLP-Models/GermanWave2Vec'

# Use the Model from this Directory (Base: .../NLP-Models/GermanWave2Vec/)
# None -> Start from 'facebook/wav2vec2-large-xlsr-53-german'
trained_model_path = f'{model_dir}/{trained_model_directory}'
git_views_dir = f'{local_nlp_base_dir}/gitviews'

if not os.path.isdir(git_views_dir):
    !mkdir $git_views_dir
    !cd $git_views_dir; git clone https://github.com/ElUnrast/GermanWave2Vec.git

runs on colab: True
mkdir: cannot create directory ‘/root/shared’: File exists
mkdir: cannot create directory ‘/root/shared/NLP-Data’: File exists


In [3]:
if runs_on_colab:
    git_view_path = f'{git_views_dir}/GermanWave2Vec'
    !cd $git_view_path; git fetch --all; git reset --hard origin/main

script_path = f'{git_views_dir}/GermanWave2Vec/python'
# Die Klassen aus GitHub sin in diesem Notebook inline definiert
# sys.path.insert(0, script_path)

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/ElUnrast/GermanWave2Vec
   218586c..9f3ddfc  main       -> origin/main
HEAD is now at 9f3ddfc Erstellt mit Colaboratory


## 2. Check Runtime Properties

In [4]:
if 'cuda' == device:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)

    if runs_on_colab:
        if wait_for_v100 and not gpu_info.find('V100') >= 0:
            print('The current GPU is not a V100')
            print('Since you want to wait for a V100 the current session is aborted')
            raise ValueError

        if gpu_info.find('failed') >= 0:
            print('For training, please use a VM with GPU!')
            raise ValueError
            
        from psutil import virtual_memory
        ram_gb = virtual_memory().total / 1e9
        print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

        if ram_gb < 20:
            print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
            print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
            print('re-execute this cell.')
        else:
            print('You are using a high-RAM runtime!')
            
    print(gpu_info)

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.
Fri May 14 15:22:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               | 

## 3 Install packages and do Imports

In [5]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import torch
import torchaudio
from torch import nn
from torch.cuda.amp import autocast

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers.trainer_pt_utils import LengthGroupedSampler, DistributedLengthGroupedSampler

import json
import collections
import librosa
import numpy as np
import pandas as pd
import sklearn
import jiwer
from jiwer import wer
from datasets import load_metric
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split

# from SnippetDatasets import SnippetDatasets
# from GermanSpeechToTextTranslater import GermanSpeechToTextTranslater

In [6]:
# Kopie aus github 'SnippetDatasets.py'
import os
import glob
import pandas as pd
import jiwer
from jiwer import wer
from zipfile import ZipFile


# Known Snippet Directories with content.csv, content-with_original.csv
class SnippetDatasets:
    def __init__(self, run_on_colab, local_audio_base_dir, git_repository=None, extern_audio_base_dir=None):
        self.local_datasets = {}
        self.extern_datasets = {}
        self.used_datasets = []
        self.local_audio_base_dir = local_audio_base_dir
        self.extern_audio_base_dir = extern_audio_base_dir
        self.git_repository = git_repository

        for dir in self.directories_with_content(local_audio_base_dir):
            self.local_datasets[os.path.basename(dir)] = dir

        if extern_audio_base_dir:
            for zip_file in glob.glob(f'{extern_audio_base_dir}/*.zip'):
                ds_id = os.path.basename(zip_file)[0:-4]
                self.extern_datasets[ds_id] = zip_file

    def directories_with_content(self, dir):
        result = []

        if self.has_content(dir) or self.has_content_original(dir):
            result.append(dir)
        else:
            sub_dir = [f'{dir}/{d}' for d in os.listdir(dir) if os.path.isdir(f'{dir}/{d}')]

            for d in sub_dir:
                result.extend(self.directories_with_content(d))

        return result

    def use_datasets(self, datasets_to_use):
        if isinstance(datasets_to_use, list):
            for ds in datasets_to_use:
                self.use_dataset(ds)
        else:
            self.use_dataset(datasets_to_use)

    def use_dataset(self, ds_id_to_use):
        if ds_id_to_use in self.local_datasets:
            self.used_datasets.append(ds_id_to_use)
        elif ds_id_to_use in self.extern_datasets:
            # TODO: Falls nicht genug Speicher da ist sollten nicht verwendete externe Datasets,
            #       welche lokal vorhanden sind, lokal gelöscht werden.
            zip_file_name = f'{self.extern_audio_base_dir}/{ds_id_to_use}.zip'
            print(f'Download and extract {zip_file_name} from gdrive')

            with ZipFile(zip_file_name, 'r') as zip:
                zip.extractall(self.local_audio_base_dir)

            self.local_datasets[ds_id_to_use] = f'{self.local_audio_base_dir}/{ds_id_to_use}'
            self.used_datasets.append(ds_id_to_use)

    def has_content(self, id_or_directory):
        return os.path.isfile(f'{self._get_directory(id_or_directory)}/content.csv')

    def has_content_original(self, id_or_directory):
        return os.path.isfile(f'{self._get_directory(id_or_directory)}/content-with_original.csv')

    def has_translation(self, id_or_directory):
        return os.path.isfile(f'{self._get_directory(id_or_directory)}/content-translated.csv')

    def has_translation_with_original(self, id_or_directory):
        return os.path.isfile(f'{self._get_directory(id_or_directory)}/content-translated-with_original.csv')

    def needs_translation(self, directory):
        if self.has_content(directory):
            return not self.has_translation(directory)
        else:
            return not self.has_translation_with_original(directory)

    def get_snippet_directory(self, ds_id):
        return self.local_datasets[ds_id]

    def get_ds_git_directory(self, ds_id):
        if self.git_repository:
            snippet_directory = get_snippet_directory(ds_id)
            
            if snippet_directory:
                return f'{self.git_repository}/datasets/{snippet_directory[len(self.local_audio_base_dir):]}'
        
        return None;

    def load_ds_content(self, id_or_directory):
        # Action: translate -> find original
        return self._get_dataframe(id_or_directory, 'content.csv')

    def load_ds_content_with_original(self, id_or_directory):
        # Action: translate -> train
        return self._get_dataframe(id_or_directory, 'content-with_original.csv')

    def load_ds_content_translated(self, id_or_directory):
        # File for Action: find original
        return self._get_dataframe(id_or_directory, 'content-translated.csv')

    def load_ds_content_translated_with_original(self, id_or_directory):
        # File for Action: train or repeated translation
        return self._get_dataframe(id_or_directory, 'content-translated-with_original.csv')
    
    def _get_directory(self, id_or_directory):
        if id_or_directory in self.local_datasets:
            return self.get_snippet_directory(id_or_directory)
        
        return id_or_directory
    
    def _get_dataframe(self, id_or_directory, file_name):
        print(f'Loading Dataset: {id_or_directory} - {file_name}')
        self.use_dataset(id_or_directory)
        ds_directory = self._get_directory(id_or_directory)
        pandas_df = pd.read_csv(f'{ds_directory}/{file_name}', sep=';')
        truncated_ds = pandas_df
        
        if 'OriginalText' in pandas_df.columns:
            print(f'Pruning Dataset {id_or_directory} with {pandas_df.shape[0]} Entries')

            if 'Length' in pandas_df.columns:
                truncated_ds = truncated_ds[(truncated_ds.Length <= 4000) & (truncated_ds.Length >= 31)]
                print(f' - {truncated_ds.shape[0]} Entries left after Length Cut (min=31, max=4000)')

            if 'Action' in pandas_df.columns:
                truncated_ds = truncated_ds[~truncated_ds.Action.str.startswith('exclude')]
                print(f' - {truncated_ds.shape[0]} Entries left after Action Cut')
        else:
            if pandas_df.Length.max() > (1600 * 3):
                raise ValueError
            
        if pandas_df.shape[0] != truncated_ds.shape[0]:
            print(f'Dataset was truncated from {pandas_df.shape[0]} to {truncated_ds.shape[0]} Entries. Saving Backup.')
            pandas_df.to_csv(f'{ds_directory}/original-{file_name}', sep=';', index=False)
            truncated_ds.to_csv(f'{ds_directory}/{file_name}', sep=';', index=False)
        
        pandas_df = truncated_ds
        return pandas_df


def calc_wer(ds_with_translation_and_original, chunk_size=1000):
    if 'Translated1' in ds_with_translation_and_original.columns:
        translation_column = ds_with_translation_and_original.Translated1
    else:
        translation_column = ds_with_translation_and_original.Translated0

    return chunked_wer(
        targets=ds_with_translation_and_original.OriginalText.tolist(),
        predictions=translation_column.tolist(),
        chunk_size=chunk_size
    )

# Chunked version, see https://discuss.huggingface.co/t/spanish-asr-fine-tuning-wav2vec2/4586/5:
def chunked_wer(targets, predictions, chunk_size=1000):
    if chunk_size is None:
        return jiwer.wer(targets, predictions)

    start = 0
    end = chunk_size
    H, S, D, I = 0, 0, 0, 0

    while start < len(targets):
        chunk_metrics = jiwer.compute_measures(targets[start:end], predictions[start:end])
        H = H + chunk_metrics["hits"]
        S = S + chunk_metrics["substitutions"]
        D = D + chunk_metrics["deletions"]
        I = I + chunk_metrics["insertions"]
        start += chunk_size
        end += chunk_size

    return float(S + D + I) / float(H + S + D)


In [7]:
# Kopie aus github 'GermanSpeechToTextTranslater.py'
import os
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import torch
import torchaudio
from torch import nn
from torch.cuda.amp import autocast

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers.trainer_pt_utils import LengthGroupedSampler, DistributedLengthGroupedSampler

import json
import collections
import librosa
import numpy as np
import pandas as pd
import sklearn
import jiwer
from jiwer import wer
from datasets import load_metric
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split


class GermanSpeechToTextTranslater:
    def __init__(
            self,
            model=None,
            processor=None,
            # language_tool=None,
            # resampled_dir=None,
            model_name=None,
            default_model_name='facebook/wav2vec2-large-xlsr-53-german',
            device='cuda'
    ):
        self.device = device
        self.model_name = model_name if model_name else default_model_name
        print(f'Using Model: {self.model_name}')
        print('Loading processor')
        # Anlegen eines eigenen Processors, da in Wav2Vec2Processor.from_pretrained(facebook/wav2vec2-large-xlsr-53-german)
        # kein ß enthalten ist
        self.my_processor = processor if processor else self.create_processor()
        print('Loading metric')
        self.my_metric = load_metric('wer')
        self.trained_epochs = 0
        
        # if os.path.isfile(f'{self.model_name}/pytorch_model.bin'):
        #     if os.path.isfile(f'{self.model_name}/trained_epochs.json'):
        #         with open(f'{self.model_name}/trained_epochs.json', 'r') as json_file:
        #             self.trained_epochs = json.load(json_file)['trained_epochs']
        #     else:
        #         with open(f'{self.model_name}/trained_epochs.json', 'w') as json_file:
        #             json.dump({'trained_epochs' : self.trained_epochs}, json_file)
        
        # TODO: in abgeleitete Klasse verlagern
        # print('Loading language tool')
        # self.my_tool = language_tool if language_tool else language_tool_python.LanguageTool('de-DE')
        print('Loading model')

        if model:
            self.my_model = model
        else:
            if model_name:
                self.my_model = Wav2Vec2ForCTC.from_pretrained(self.model_name).to(device)
            else:
                self.my_model = Wav2Vec2ForCTC.from_pretrained(
                    self.model_name,
                    # activation_dropout=0.055
                    attention_dropout=0.1,  # 0.094
                    hidden_dropout=0.1,  #
                    feat_proj_dropout=0.0,  # 0.04
                    mask_time_prob=0.05,  # 0.08
                    layerdrop=0.1,  # 0.04
                    gradient_checkpointing=True,
                    ctc_loss_reduction="mean",
                    pad_token_id=self.my_processor.tokenizer.pad_token_id,
                    vocab_size=len(self.my_processor.tokenizer)
                ).to(device)
        self.my_model.freeze_feature_extractor()

    def reload_from_checkpoint(self, checkpoint):
        del self.my_model
        torch.cuda.empty_cache()
        self.model_name = checkpoint
        print(f'Using Model: {self.model_name}')
        self.my_model = Wav2Vec2ForCTC.from_pretrained(self.model_name).to(device)
        torch.cuda.empty_cache()
        self.my_model.freeze_feature_extractor()

    # def translate(self, audio_file_name):
    #    translation = self.translate_audio(audio_file_name)
    #    return self.my_tool.correct(translation), translation

    def load_as_sr16000(self, audio_file_name):
        new_path = None

        if not audio_file_name.endswith('.mp3'):
            samples, sampling_rate = librosa.load(audio_file_name, sr=16_000)  # Downsample to 16kHz
        else:
            # print( f'load {audio_file_name}')
            samples, sampling_rate = torchaudio.load(audio_file_name)
            # print(f'samples.shape : {samples.shape}, sampling_rate : {sampling_rate}')
            samples = samples[0]

            if sampling_rate != 16_000:
                # print( f'Converting from {sampling_rate}')
                samples = librosa.resample(np.asarray(samples), sampling_rate, 16_000)
                # funktioniert auch:
                # samples = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16_000)(samples).squeeze().numpy()

        if samples.shape[0] == 2:
            samples = samples[0]

        if isinstance(samples, (np.ndarray, np.generic)):
            samples = torch.from_numpy(samples).float().flatten()
        else:
            samples = samples.squeeze().numpy()

        if new_path != None:
            torch.save(samples, new_path)

        samples_size = samples.shape[0]
        return samples, samples.shape[0]

    def audio_to_cuda_inputs(self, audio_file_name):
        samples, samples_size = self.load_as_sr16000(audio_file_name)
        return self.my_processor(samples, return_tensors="pt", sampling_rate=16_000).input_values, samples_size

    def translate_audio(self, audio_file_name):
        samples, samples_size = self.audio_to_cuda_inputs(audio_file_name)
        samples_size = samples_size

        with torch.no_grad():
            logits = self.my_model(samples.to(self.device)).logits

        # Storing predicted ids
        predicted_ids = torch.argmax(logits, dim=-1)
        # Converting audio to text - Passing the prediction to the tokenzer decode to get the transcription
        return self.my_processor.decode(predicted_ids[0]), samples_size

    def translate_and_extend_dataset_from_directory(self, ds_loader, id_or_directory):
        if not ds_loader.needs_translation(id_or_directory):
            return

        print(f'Translating and extend Dataset: {id_or_directory}')
        has_original = ds_loader.has_content_original(id_or_directory)

        if not has_original:
            ds = ds_loader.load_ds_content(id_or_directory)
        else:
            ds = ds_loader.load_ds_content_with_original(id_or_directory)

        ds_dir_name = ds_loader.get_snippet_directory(id_or_directory)
        translated_list, size_list = self.translate_dataset(ds_dir_name, ds)

        if 'Translated1' in ds.columns:
            ds['Size'] = size_list
            ds['Translated0'] = translated_list
            del ds['Translated1']
        elif 'Translated0' in ds.columns:
            ds['Size'] = size_list
            ds['Translated0'] = translated_list
        else:
            ds.insert(loc=8, column='Size', value=size_list)
            ds.insert(loc=9, column='Translated0', value=translated_list)

        if not has_original:
            ds.to_csv(f'{ds_dir_name}/content-translated.csv', sep=';', index=False)
        else:
            ds.to_csv(f'{ds_dir_name}/content-translated-with_original.csv', sep=';', index=False)

    def translate_dataset(self, mp3_dir, ds):
        if isinstance(ds, GermanTrainingWav2Vec2Dataset):
            files = [f'{mp3_dir}/{file_name}' for file_name in ds.paths]
        else:
            files = [f'{mp3_dir}/{file_name}' for file_name in ds.Datei]

        return self.translate_audio_files(files)

    def translate_audio_files(self, files):
        translated_list = []
        size_list = []
        idx = 0

        for file_name in tqdm_notebook(files):
            if (idx % 20) == 0:
                torch.cuda.empty_cache()

            idx = idx + 1
            translated, sample_size = self.translate_audio(file_name)
            translated_list.append(translated)
            size_list.append(sample_size)

        return translated_list, size_list

    def split_dataset(
            self,
            pandas_df,
            max_trainingset_size=25000,
            max_sample_size=1000,
            use_only_incorrect_translated=True
    ):
        used_df = pandas_df
        has_action = 'Action' in pandas_df.columns

        if has_action:
            used_df = used_df[(used_df.Action == 'train') | (used_df.Action == 'translate')]
            print(f' - {used_df.shape[0]} Entries left after Action Cut')

        used_df = used_df[used_df.Length <= max_sample_size] if max_sample_size else pandas_df
        print(f' - {used_df.shape[0]} Entries left after Length Cut (max={max_sample_size})')
        used_df = used_df[used_df.Length > 30]
        print(f' - {used_df.shape[0]} Entries left after minimal Size Cut (min=31)')

        # Test Dataset should be fixed_training_set_size Entries long
        test_percentage = fixed_training_set_size / used_df.shape[0] if fixed_training_set_size else 0.2

        train, test = train_test_split(used_df, test_size=test_percentage, random_state=143)
        print(f'Training Dataset Size: {train.shape[0]}, Validation Dataset Size {test.shape[0]}')

        if use_only_incorrect_translated:
            if 'Translated1' in train.columns:
                train = train[(train.OriginalText != train.Translated1)]
            else:
                train = train[(train.OriginalText != train.Translated0)]

            print(f' - {train.shape[0]} Entries left after first Recognition Cut')

        train = sklearn.utils.shuffle(train)

        if max_trainingset_size:
            train = train[:min(train.shape[0], max_trainingset_size)]
            print(f' - {train.shape[0]} left after Entries Max Samples Cut (max={max_trainingset_size})')

        print(f'Training Dataset Size: {train.shape[0]}, Validation Dataset Size {test.shape[0]}')
        return train, test

    def get_trainer(
            self,
            training_args,
            snippet_directory,
            train_ds,
            test_ds,
            use_grouped_legth_trainer=False
    ):
        train_dataset = GermanTrainingWav2Vec2Dataset(self, snippet_directory, train_ds, 'train')
        test_dataset = GermanTrainingWav2Vec2Dataset(self, snippet_directory, test_ds, 'eval') if not test_ds.empty else None
        data_collator = DataCollatorCTCWithPadding(processor=self.my_processor, padding=True)

        if use_grouped_legth_trainer:
            trainer = GroupedLengthsTrainer(
                model=self.my_model,
                data_collator=data_collator,
                args=training_args,
                compute_metrics=self.compute_metrics,
                train_dataset=train_dataset,
                eval_dataset=test_dataset,
                tokenizer=self.my_processor.feature_extractor,
                train_seq_lengths=train_dataset.input_seq_lengths
            )
        else:
            trainer = Trainer(
                model=self.my_model,
                data_collator=data_collator,
                args=training_args,
                compute_metrics=self.compute_metrics,
                train_dataset=train_dataset,
                eval_dataset=test_dataset,
                tokenizer=self.my_processor.feature_extractor,
            )

        return trainer

    def create_processor(self):
        vocab_file_name = 'vocab.json'

        if not os.path.isfile(vocab_file_name):
            vocab_dict = {
                '<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4,
                'e': 5, 'n': 6, 'i': 7, 's': 8, 'r': 9, 't': 10, 'a': 11,
                'h': 12, 'd': 13, 'u': 14, 'l': 15, 'c': 16, 'g': 17, 'm': 18,
                'o': 19, 'b': 20, 'w': 21, 'f': 22, 'k': 23, 'z': 24, 'v': 25,
                'ü': 26, 'p': 27, 'ä': 28, 'ö': 29, 'j': 30, 'y': 31, "'": 32,
                'x': 33, 'q': 34, 'ß': 35
            }

            with open(vocab_file_name, 'w') as vocab_file:
                json.dump(vocab_dict, vocab_file)

        tokenizer = Wav2Vec2CTCTokenizer(
            "vocab.json",
            unk_token="<unk>",
            pad_token="<pad>",
            word_delimiter_token="|"
        )
        feature_extractor = Wav2Vec2FeatureExtractor(
            feature_size=1, sampling_rate=16_000, padding_value=0.0, do_normalize=True, return_attention_mask=True
        )
        return Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

    def test(self, ds_id, pandas_df, mp3_dir, diff_file_extension, diff_calc_wer=False):
        if not 'Autor' in pandas_df.columns:
            raise ValueError

        wer_result = 1.0
        translation_column_name = 'Translated1'

        print(f'Translate all')
        predictions, _ = self.translate_dataset(mp3_dir, pandas_df)
        pandas_df[translation_column_name] = predictions
        pandas_df.to_csv(f'{mp3_dir}/content-translated-with_original.csv', sep=';', index=False)

        if diff_calc_wer:
            print('Calculate WER')
            wer_result = self.calc_wer(pandas_df)
            print(f'WER: {wer_result}')

        print('Saving diff files')
        truncated_ds = pandas_df[
            ((pandas_df.Action == 'train') | (pandas_df.Action == 'translate')) & (pandas_df.Length <= 1200)
        ]
        bad_translation_ds = truncated_ds[truncated_ds[translation_column_name] != truncated_ds['OriginalText']]

        translations = bad_translation_ds[translation_column_name].tolist()
        original_texts = bad_translation_ds['OriginalText'].tolist()
        file_names = bad_translation_ds['Datei'].tolist()

        orig_file = open(f'{model_dir}/{ds_id}-original-{diff_file_extension}.txt', 'w')
        translated_file = open(f'{model_dir}/{ds_id}-translated-{diff_file_extension}.txt', 'w')

        for file_name, original_text, translation in zip(file_names, original_texts, translations):
            orig_file.write(f'{file_name}, {original_text}\n')
            translated_file.write(f'{file_name}, {translation}\n')

        orig_file.close()
        translated_file.close()

        return bad_translation_ds, truncated_ds, wer_result

    def train(
        dataset_loader, 
        ds_to_train, 
        max_trainingset_size, 
        max_rounds, 
        num_train_epochs,
        num_steps_per_epoche, # max_steps,
        per_device_train_batch_size,
        per_device_eval_batch_size,
        gradient_accumulation_steps,
        logging_steps,
        learning_rate,
        warmup_steps,
        early_stopping=0.2
    ):
        training_args = TrainingArguments(
            output_dir=trained_model_path,
            group_by_length=True,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_eval_batch_size, 
            gradient_accumulation_steps=gradient_accumulation_steps, 
            evaluation_strategy="steps",
            max_steps=num_steps_per_epoche,         
            fp16=True,
            # save_steps=save_steps,     
            # eval_steps=eval_steps,     
            logging_steps=logging_steps, 
            learning_rate=learning_rate,
            warmup_steps=warmup_steps,   
            # save_total_limit=2
        )

        for runde in range(max_rounds):
            print(f'Starting round {runde} of {max_rounds}')

            for ds_id in ds_to_train:
                if not dataset_loader.has_translation_with_original(ds_id):
                    ds_to_train.remove(ds_id)
                    continue

                WANDB_NOTES=ds_id
                pandas_df = dataset_loader.load_ds_content_translated_with_original(ds_id)
                print(f'Dataset - {ds_id} loaded with {pandas_df.shape[0]} Entries')
                mp3_dir = dataset_loader.get_snippet_directory(ds_id)
              
                for epoche in range(num_train_epochs):
                    print(f'Starting round {runde} of {max_rounds}, epoche {epoche} of {num_train_epochs}')
                    print(f'Splitting Dataset {ds_id} with {pandas_df.shape[0]} Entries')
                    bad_translation_ds, truncated_ds, wer_result = translator.test(
                        ds_id, 
                        pandas_df, 
                        mp3_dir, 
                        diff_file_extension=f'{epoche}-{runde}', 
                        diff_calc_wer=True
                    )
                  
                    if (bad_translation_ds.shape[0] > (truncated_ds.shape[0] * early_stopping)) and (wer_result < early_stopping):                    
                        if max_trainingset_size:
                            train = bad_translation_ds[:min(bad_translation_ds.shape[0], max_trainingset_size)]
                            print(f' - {train.shape[0]} left after Entries Max Samples Cut (max={max_trainingset_size})')
                      
                        print(f'Creating Trainer for {ds_id}')
                        trainer = translator.get_trainer(
                            training_args, 
                            mp3_dir,             
                            train_pandas_ds,
                            None,  # test_pandas_ds,
                            use_grouped_legth_trainer=False
                        )
                        print(f'Training of Dataset: {ds_id}')
                        train_result = trainer.train()

                        print(f'Save Model')
                        trainer.save_model()
                        metrics = train_result.metrics
                        max_train_samples = train_pandas_ds.shape[0]
                        metrics["train_samples"] = min(max_train_samples, train_pandas_ds.shape[0])
                        trainer.log_metrics("train", metrics)
                        trainer.save_metrics("train", metrics)
                        trainer.save_state()
                        torch.cuda.empty_cache()
                        self.trained_epochs = self.trained_epochs + 1
                        with open(f'{self.model_name}/trained_epochs.json', 'w') as json_file:
                            json.dump({'trained_epochs', self.trained_epochs}, json_file)
                    else:
                        # mindestens 98% der Sätze wurde korrekt übersetzt. Überprüfung der Problemfälle ist angebracht.
                        # Es hat sich gezeigt, dass das Ergebnis wieder schlechter werden kann.
                        print('Early stopping!')
                        ds_to_train.remove(ds_id)
                        break

                print(f'final check und update of {ds_id}')
                bad_translation_ds, truncated_ds, wer_result = translator.test(
                    ds_id, 
                    pandas_df, 
                    mp3_dir, 
                    diff_file_extension=f'{epoche}-{runde}', 
                    diff_calc_wer=True
                )

        print('Training finisched!')
    
    def compute_metrics(self, pred):
        # we do not want to group tokens when computing the metrics
        label_str = self.my_processor.batch_decode(pred.label_ids, group_tokens=False)
        pred_logits = pred.predictions
        pred_ids = np.argmax(pred_logits, axis=-1)
        pred.label_ids[pred.label_ids == -100] = self.my_processor.tokenizer.pad_token_id
        pred_str = self.my_processor.batch_decode(pred_ids)

        # Bugs: strings in the list of strings 'pred_str' have 2 extra characters ' ' and first character in vocab_dict
        #       strings in the list of strings 'label_str' have extra charecters '[UNK]'
        # Correction:
        # pred_str = [item[:-2] for item in pred_str]

        def g(s):
            s = s.strip()

            while s[-5:] == "<unk>":
                s = s[:-5]

            return s

        label_str_c = [g(item) for item in label_str]

        # def f(s):
        #     if len(s) <= 2:    # maybe 2 -> 1 later
        #         return s
        #     else:
        #         return s[:-2]   # maybe 2 -> 1 later

        # pred_str_c = [f(item) for item in pred_str]
        pred_str_c = pred_str
        # print(f'"{label_str_c}" - "{pred_str_c}"')
        return {"wer": jiwer.compute_measures(label_str_c, pred_str_c)["wer"]}


class GermanTrainingWav2Vec2Dataset(torch.utils.data.Dataset):
    def __init__(self, german_speech_translator, snippet_directory, ds, split):
        super().__init__()
        assert split in {'train', 'eval'}
        self.split = split
        self.snippet_directory = snippet_directory
        self.german_speech_translator = german_speech_translator
        self.max_input_length_quantile = .98
        self.max_input_length = None

        if split == 'train':
            self.input_seq_lengths = ds['Size'].tolist()
            self.max_input_length = torch.tensor(self.input_seq_lengths).float() \
                .quantile(self.max_input_length_quantile).int().item()

        self.labels = ds['OriginalText'].tolist()
        self.paths = ds['Datei'].tolist()

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        mp3_file = f'{self.snippet_directory}/{self.paths[idx]}'
        # print(f'Loading MP3: {mp3_file}')
        inputs, _ = self.german_speech_translator.audio_to_cuda_inputs(
            mp3_file
        )

        inputs = inputs.squeeze()
        # print( f'Cuda Inputs created. {type(inputs)}, {inputs.shape}')
        if self.split == 'train':
            inputs = inputs[:self.max_input_length]

        label_str = self.labels[idx]
        # print(f'Label: {type(label_str)}')

        processor = self.german_speech_translator.my_processor

        with processor.as_target_processor():
            label = processor(label_str).input_ids

        return {'input_values': inputs, 'labels': label}


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # print(f'Padding input: {self.padding}')
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


# solution from https://discuss.huggingface.co/t/spanish-asr-fine-tuning-wav2vec2/4586/6
class GroupedLengthsTrainer(Trainer):
    # length_field_name should possibly be part of TrainingArguments instead
    def __init__(self, train_seq_lengths: List[int], *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.train_seq_lengths = train_seq_lengths

    def _get_train_sampler(self) -> Optional[torch.utils.data.sampler.Sampler]:
        if isinstance(self.train_dataset, torch.utils.data.IterableDataset) or not isinstance(
                self.train_dataset, collections.abc.Sized
        ):
            return None

        # Build the sampler.
        if self.args.group_by_length:
            # lengths = self.train_dataset[self.length_field_name] if self.length_field_name is not None else None
            model_input_name = self.tokenizer.model_input_names[0] if self.tokenizer is not None else None

            if self.args.world_size <= 1:
                print('Using LengthGroupedSampler')
                return LengthGroupedSampler(
                    self.train_dataset, self.args.train_batch_size, lengths=self.train_seq_lengths,
                    model_input_name=model_input_name
                )
            else:
                print('Using DistributedLengthGroupedSampler')
                return DistributedLengthGroupedSampler(
                    self.train_dataset,
                    self.args.train_batch_size,
                    num_replicas=self.args.world_size,
                    rank=self.args.process_index,
                    lengths=self.train_seq_lengths,
                    model_input_name=model_input_name,
                )

        else:
            return super()._get_train_sampler()

## 4 Initialize Helper Classes

In [8]:
my_datasets = SnippetDatasets(
    runs_on_colab, 
    local_audio_base_dir=local_data_dir, 
    extern_audio_base_dir=extern_data_dir
)

In [9]:
model_path = None

if checkpoint:
    model_path = f'{trained_model_path}/{checkpoint}'
elif os.path.isfile(f'{trained_model_path}/pytorch_model.bin'):
    model_path = trained_model_path

print(f'Initialize Model from Path: {model_path}')
translator = GermanSpeechToTextTranslater(model_name=model_path)

Initialize Model from Path: /content/gdrive/MyDrive/NLP-Models/GermanWave2Vec/trained_model_ah
Using Model: /content/gdrive/MyDrive/NLP-Models/GermanWave2Vec/trained_model_ah
Loading processor
Loading metric
Loading model


In [10]:
def train():
  training_args = TrainingArguments(
      output_dir=trained_model_path,
      group_by_length=True,
      per_device_train_batch_size=per_device_train_batch_size,
      per_device_eval_batch_size=per_device_eval_batch_size, 
      gradient_accumulation_steps=gradient_accumulation_steps, 
      evaluation_strategy="steps",
      max_steps=max_steps,         
      fp16=True,
      save_steps=save_steps,     
      eval_steps=eval_steps,     
      logging_steps=logging_steps, 
      learning_rate=learning_rate,
      warmup_steps=warmup_steps,   
      save_total_limit=2
  )

  ds_to_train = []

  for ds in get_used_datasets():
      print(f'Use Dataset: {ds}')
      ds_to_train.append(ds)
      
  calc_wer_after_training = False  # Wegen OutOfMemory ausgeschaltet 

  for runde in range(max_rounds):
      print(f'Starting round {runde} of {max_rounds}')

      for ds_id in ds_to_train:
          # if not my_datasets.has_translation_with_original(ds_id):
          #    print(f'Removing Dataset: {ds_id}')
          #    ds_to_train.remove(ds_id)
          #    continue

          WANDB_NOTES=ds_id
          stop = False
          pandas_df = my_datasets.load_ds_content_translated_with_original(ds_id)
          print(f'Dataset - {ds_id} loaded with {pandas_df.shape[0]} Entries')
          mp3_dir = my_datasets.get_snippet_directory(ds_id)
          
          for epoche in range(num_train_epochs):
              print(f'Starting round {runde} of {max_rounds}, epoche {epoche} of {num_train_epochs}')
              print(f'Splitting Dataset {ds_id} with {pandas_df.shape[0]} Entries')
              train_pandas_ds, test_pandas_ds = translator.split_dataset(
                  pandas_df, 
                  max_trainingset_size=max_trainingset_size, 
                  max_sample_size=max_sample_size
              )
              
              if train_pandas_ds.shape[0] > pandas_df.shape[0] * 0.02:            
                  print(f'Creating Trainer for {ds_id}')
                  trainer = translator.get_trainer(
                      training_args, 
                      mp3_dir,             
                      train_pandas_ds,
                      test_pandas_ds,
                      use_grouped_legth_trainer=False
                  )
                  print(f'Training of Dataset: {ds_id}')
                  train_result = trainer.train()

                  print(f'Save Model')
                  trainer.save_model()
                  metrics = train_result.metrics
                  max_train_samples = train_pandas_ds.shape[0]
                  metrics["train_samples"] = min(max_train_samples, train_pandas_ds.shape[0])
                  trainer.log_metrics("train", metrics)
                  trainer.save_metrics("train", metrics)
                  trainer.save_state()
                  torch.cuda.empty_cache()

                  last_checkpoint = get_last_checkpoint(trained_model_path)
                  print( f'last Checkpoint: {last_checkpoint}')
                  
                  # print(f'Reload from last Checkpoint: {last_checkpoint}')
                  # translator.reload_from_checkpoint(last_checkpoint)
                  # {last_checkpoint}-
                  bad_translation_ds, truncated_ds, wer_result = translator.test(
                      ds_id, 
                      pandas_df, 
                      mp3_dir, 
                      diff_file_extension=f'{epoche}-{runde}', 
                      diff_calc_wer=calc_wer_after_training
                  )
                  
                  if (bad_translation_ds.shape[0] < (truncated_ds.shape[0] * 0.02)) or (wer_result < 0.02):
                      # mindestens 98% der Sätze wurde korrekt übersetzt. Überprüfung der Problemfälle ist angebracht.
                      # Es hat sich gezeigt, dass das Ergebnis wieder schlechter werden kann.
                      stop = True
              else:
                  stop = True

              if stop:
                  print('Early stopping!')
                  ds_to_train.remove(ds_id)
                  break

  print('Training finisched!')

## 4. Choose Dataset

In [11]:
all_ds_ids = set()
all_ds_ids.update(list(my_datasets.local_datasets.keys()))
all_ds_ids.update(list(my_datasets.extern_datasets.keys()))

import difflib
from functools import partial

import ipywidgets as widgets
from IPython.display import display, Audio

ds_checkbox_items = []

for ds_id in sorted(all_ds_ids):
    ds_checkbox_items.append(widgets.Checkbox(
        value=ds_id in ds_to_us, 
        description=ds_id, 
        disabled=False, 
        indent=False
    ))

def get_used_datasets(checkbox_items=ds_checkbox_items):
    result = []

    for item in checkbox_items:
        if item.value:
            result.append(item.description)

    return result

print('Please choose Datasets to use')
ds_checkboxes_widget = widgets.Box(ds_checkbox_items)
display(ds_checkboxes_widget)

Please choose Datasets to use


Box(children=(Checkbox(value=True, description='HP1-FvM', indent=False), Checkbox(value=False, description='HP…

In [ ]:
train()

Use Dataset: HP1-FvM
Starting round 0 of 1
Loading Dataset: HP1-FvM - content-translated-with_original.csv
Pruning Dataset HP1-FvM with 8055 Entries
 - 8055 Entries left after Length Cut (min=31, max=4000)
 - 8055 Entries left after Action Cut
Dataset - HP1-FvM loaded with 8055 Entries
Starting round 0 of 1, epoche 0 of 1
Splitting Dataset HP1-FvM with 8055 Entries
 - 8055 Entries left after Action Cut
 - 8055 Entries left after Length Cut (max=1200)
 - 8055 Entries left after minimal Size Cut (min=31)
Training Dataset Size: 7855, Validation Dataset Size 200
 - 3782 Entries left after first Recognition Cut
 - 2500 left after Entries Max Samples Cut (max=2500)
Training Dataset Size: 2500, Validation Dataset Size 200
Creating Trainer for HP1-FvM
Training of Dataset: HP1-FvM


In [ ]:
def on_train_action_button_clicked(b):
    print('Train')
    train()

def on_translate_action_button_clicked(b):
    print('Translate')
    for ds_id in get_used_datasets():
        translator.translate_and_extend_dataset_from_directory(my_datasets, ds_id)    

class InvisibleAudio(Audio):
    def _repr_html_(self):
        audio = super()._repr_html_()
        audio = audio.replace('<audio', f'<audio onended="this.parentNode.removeChild(this)"')
        return f'<div style="display:none">{audio}</div>'

def show_diff(text, n_text):
    """
    http://stackoverflow.com/a/788780
    Unify operations between two compared strings seqm is a difflib.
    SequenceMatcher instance whose a & b are strings
    """
    seqm = difflib.SequenceMatcher(None, text, n_text)
    output= []
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'equal':
            output.append(seqm.a[a0:a1])
        elif opcode == 'insert':
            output.append("<font color=red>^" + seqm.b[b0:b1] + "</font>")
        elif opcode == 'delete':
            output.append("<font color=blue>^" + seqm.a[a0:a1] + "</font>")
        elif opcode == 'replace':
            # seqm.a[a0:a1] -> seqm.b[b0:b1]
            output.append("<font color=green>^" + seqm.b[b0:b1] + "</font>")
        else:
            raise RuntimeError("unexpected opcode")
    return ''.join(output)
    
def create_diff_row(audio_file, original_text, translated_text ):
    diff_widget = widgets.HTML(value=show_diff(original_text, translated_text))
    button_widget = widgets.Button(description='Play')

    def on_play_button_clicked(f, b):
        display(InvisibleAudio(url=audio_file, autoplay=True))

    button_widget.on_click(partial(on_play_button_clicked, 1))
    return widgets.HBox([button_widget, diff_widget])

def create_diff_content(ds_id):
    ds = my_datasets.load_ds_content_translated_with_original(ds_id)
    snipped_directory = my_datasets.get_snippet_directory(ds_id)
    print(f'Use Snipped Directory: {snipped_directory}')
    translation_row = 'Translated1' if 'Translated1' in ds.columns else 'Translated0'
    ds = ds[ds['OriginalText'] != ds[translation_row]]
    rows = []
    max_len = min(20, len(ds))

    for idx in tqdm_notebook(range(max_len)):
        rows.append(create_diff_row(
            f'{snipped_directory}/{ds.iloc[idx]["Datei"]}', 
            ds.iloc[idx]['OriginalText'], 
            ds.iloc[idx][translation_row]
        ))

    return widgets.VBox(rows)

def on_validate_action_button_clicked(b):
    print('Validate')
    tab = widgets.Tab()
    tab_titles = []
    tab_children = []

    for ds_id in get_used_datasets():
        tab_titles.append(ds_id)
        tab_children.append(create_diff_content(ds_id))

    tab.children = tab_children
  
    for idx, name in enumerate(tab_titles):
        tab.set_title(idx, name)
  
    display(tab)


train_action_button = widgets.Button(description='Train')
train_action_button.on_click(on_train_action_button_clicked)
translate_action_button = widgets.Button(description='Translate')
translate_action_button.on_click(on_translate_action_button_clicked)
validate_action_button = widgets.Button(description='Validate')
validate_action_button.on_click(on_validate_action_button_clicked)

action_buttons = widgets.HBox([
    train_action_button, 
    translate_action_button, 
    validate_action_button
])
display(action_buttons)